In [152]:
#inf. DMRG#
import numpy as np
from scipy import integrate
import scipy.sparse.linalg.eigen.arpack as arp

In [153]:
# Параметры модели константы, размерности связей, число частиц.
J=-1.0; g=0.5; chi=5; d=2; N=5000;

In [154]:
# Инициализация начального состояние MPS - Придерживаемся канонической формы! САЙТЫ A B - записываются в один массив
B=[];s=[]
for i in range(2):
    B.append(np.zeros([2,1,1])); B[-1][0,0,0]=1
    s.append(np.ones([1]))

In [155]:
# Задание Гамильтониана, MPO, и окружения.
H_bond = np.array( [[J,g/2,g/2,0], [g/2,-J,0,g/2], [g/2,0,-J,g/2], [0,g/2,g/2,J]] )

sx = np.array([[0.,1.],[1.,0.]]); sz = np.array([[1.,0.],[0.,-1.]])
w = np.zeros((3,3,2,2),dtype=np.float)
w[0,:2] = [np.eye(2),sz]
w[0:,2] = [g*sx, J*sz, np.eye(2)]

Lp = np.zeros([1,1,3]); Lp[0,0,0] = 1.
Rp = np.zeros([1,1,3]); Rp[0,0,2] = 1.

class hamiltonian(object):
    def __init__(self,Lp,Rp,w,dtype=float):
        self.Lp = Lp
        self.Rp = Rp
        self.w = w
        self.d = w.shape[3]
        self.chia = Lp.shape[0]
        self.chib = Rp.shape[0]
        self.shape = np.array([self.d**2*self.chia*self.chib,self.d**2*self.chia*self.chib])
        self.dtype = dtype

    def matvec(self,x): #RESHAPE #
        x=np.reshape(x,(self.d,self.chia,self.d,self.chib))
        x=np.tensordot(self.Lp,x,axes=(0,1))  #Сворачиваем с окружением слева#
        x=np.tensordot(x,self.w,axes=([1,2],[0,2])) # Умножением на MPO матрицу#
        x=np.tensordot(x,self.w,axes=([3,1],[0,2])) # Умножением на MPO матрицу#
        x=np.tensordot(x,self.Rp,axes=([1,3],[0,2])) #Сворачиваем с окружением справа#
        x=np.reshape(np.transpose(x,(1,0,2,3)),((self.d*self.d)*(self.chia*self.chib)))
        if(self.dtype==float):
            return np.real(x)
        else:
            return(x)


In [156]:
# Итерации. 
for step in range(N):
    E=[]
    for i_bond in [0,1]:
        ia = np.mod(i_bond-1,2); ib = np.mod(i_bond,2); ic = np.mod(i_bond+1,2) 
        #Эта хитрая запись подходит для того, чтобы удобно задавать связь между сайтами A и B
        chia = B[ib].shape[1]; chic = B[ic].shape[2]
        
        # Свёртка n, n+1 #
        theta0 = np.tensordot(np.diag(s[ia]),np.tensordot(B[ib],B[ic],axes=(2,1)),axes=(1,1))
        theta0 = np.reshape(np.transpose(theta0,(1,0,2,3)),((chia*chic)*(d**2)))

        # Диаганолизация Гамильтониана#
        H = hamiltonian(Lp,Rp,w,dtype=float)
        e0,v0 = arp.eigsh(H,k=1,which='SA',return_eigenvectors=True,v0=theta0)
        theta = np.reshape(v0.squeeze(),(d*chia,d*chic));

        # Разложение Шмидта - SVD #
        X, Y, Z = np.linalg.svd(theta); Z = Z.T
        
        chib = np.min([np.sum(Y>10.**(-12)), chi]) #ПРОЦЕДУРА ОБРЕЗКИ - ФИКСИРУЕМ РАЗМЕРНОСТЬ MPS ! #
        X=np.reshape(X[:d*chia,:chib],(d,chia,chib))
        Z=np.transpose(np.reshape(Z[:d*chic,:chib],(d,chic,chib)),(0,2,1))

        # Обновление Окружения # - Последовательная свёртка
        Lp = np.tensordot(Lp, w, axes=(2,0))
        Lp = np.tensordot(Lp, X, axes=([0,3],[1,0]))
        Lp = np.tensordot(Lp, np.conj(X), axes=([0,2],[1,0]))
        Lp = np.transpose(Lp,(1,2,0))

        Rp = np.tensordot(w, Rp, axes=(1,2))
        Rp = np.tensordot(np.conj(Z),Rp, axes=([0,2],[2,4]))
        Rp = np.tensordot(Z,Rp, axes=([0,2],[2,3]))
        
        # Обновление MPS сайтов AB + НОРМИРОВКА #
        s[ib] = Y[:chib]/np.sqrt(sum(Y[:chib]**2))
        B[ib] = np.transpose(np.tensordot(np.diag(s[ia]**(-1)),X,axes=(1,1)),(1,0,2))
        B[ib] = np.tensordot(B[ib], np.diag(s[ib]),axes=(2,1))
        
        B[ic] = Z

In [157]:
#Вычисление Энергии#
E=[]
for i_bond in range(2):
    BB = np.tensordot(B[i_bond],B[np.mod(i_bond+1,2)],axes=(2,1))
    sBB = np.tensordot(np.diag(s[np.mod(i_bond-1,2)]),BB,axes=(1,1))
    C = np.tensordot(sBB,np.reshape(H_bond,[d,d,d,d]),axes=([1,2],[2,3]))
    E.append(np.squeeze(np.tensordot(np.conj(sBB),C,axes=([0,3,1,2],[0,1,2,3]))).item()) 
print("E_iDMRG =", np.mean(E))


f = lambda k,g : -2*np.sqrt(1+g**2-2*g*np.cos(k))/np.pi/2.  # ТОЧНОЕ РЕШЕНИЕ ! 1D Модель Изинга Беск. цепочка.
E0_exact = integrate.quad(f, 0, np.pi, args=(g,))[0]        # ТОЧНОЕ ЗНАЧЕНИЕ МИНИМАЛЬНОЙ ЭНЕРГИИ ! 
print ("E_exact = ", E0_exact)
print('dE = ', (np.mean(E) - E0_exact)/E0_exact*100)

E_iDMRG = -1.0635444099731841
E_exact =  -1.063544409973372
dE =  -1.7662613239749873e-11


In [158]:
'''НОВЫЕ ФУНКЦИИ
np.tensordot
np.item
'''

'НОВЫЕ ФУНКЦИИ\nnp.tensordot\nnp.item\n'